# Library Import

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

# CSV to DataFrame

In [ ]:
df = pd.read_csv('online_retail_data.csv')
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5.0,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1.0,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5.0,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1.0,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1.0,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
113938,505331,22550,HOLIDAY FUN LUDO,4.0,2010-04-21 12:36:00,3.75,17622.0
113939,505331,21888,BINGO SET,4.0,2010-04-21 12:36:00,3.75,17622.0
113940,505331,21791,VINTAGE HEADS AND TAILS CARD GAME,12.0,2010-04-21 12:36:00,1.25,17622.0
113941,505331,22548,HEADS AND TAILS SPORTING FUN,12.0,2010-04-21 12:36:00,1.25,17622.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113943 entries, 0 to 113942
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      113943 non-null  object 
 1   product_code  113943 non-null  object 
 2   product_name  112886 non-null  object 
 3   quantity      113942 non-null  float64
 4   order_date    113942 non-null  object 
 5   price         113942 non-null  float64
 6   customer_id   87366 non-null   float64
dtypes: float64(3), object(4)
memory usage: 6.1+ MB


# Data Cleansing

In [26]:
df_clean = df.copy()
# membuat kolom date
df_clean['date'] = pd.to_datetime(df_clean['order_date']).dt.date.astype('datetime64[ns]')
# menghapus semua baris tanpa customer_id
df_clean = df_clean[~df_clean['customer_id'].isna()]
# menghapus semua baris tanpa product_name
df_clean = df_clean[~df_clean['product_name'].isna()]
# membuat semua product_name berhuruf kecil
df_clean['product_name'] = df_clean['product_name'].str.lower()
# menghapus semua baris dengan product_code atau product_name test
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test '))]
# membuat kolom order_status dengan nilai 'cancelled' jika order_id diawali dengan huruf 'c' dan 'delivered' jika order_id tanpa awalan huruf 'c'
df_clean['order_status'] = np.where(df_clean['order_id'].str[:1]=='C', 'cancelled', 'delivered')
# mengubah nilai quantity yang negatif menjadi positif karena nilai negatif tersebut hanya menandakan order tersebut cancelled
df_clean['quantity'] = df_clean['quantity'].abs()
# menghapus baris dengan price bernilai negatif
df_clean = df_clean[df_clean['price']>0]
# membuat nilai amount, yaitu perkalian antara quantity dan price
df_clean['amount'] = df_clean['quantity'] * df_clean['price']
# mengganti product_name dari product_code yang memiliki beberapa product_name dengan salah satu product_name-nya yang paling sering muncul
most_freq_product_name = df_clean.groupby(['product_code','product_name'], as_index=False).agg(order_cnt=('order_id','nunique')).sort_values(['product_code','order_cnt'], ascending=[True,False])
most_freq_product_name['rank'] = most_freq_product_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_product_name = most_freq_product_name[most_freq_product_name['rank']==1].drop(columns=['order_cnt','rank'])
df_clean = df_clean.merge(most_freq_product_name.rename(columns={'product_name':'most_freq_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_freq_product_name']
df_clean = df_clean.drop(columns='most_freq_product_name')
# mengkonversi customer_id menjadi string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)
# menghapus outlier
from scipy import stats
df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity','amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean

<ipython-input-26-f25f6402a0c0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['order_status'] = np.where(df_clean['order_id'].str[:1]=='C', 'cancelled', 'delivered')
<ipython-input-26-f25f6402a0c0>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['quantity'] = df_clean['quantity'].abs()


,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,order_status,amount
0,C493411,21539,retro spots butter dish,1.0,2010-01-04 09:43:00,4.25,14590.0,2010-01-04,cancelled,4.25
1,493414,21844,retro spot mug,36.0,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,91.80
2,493414,21533,retro spot large milk jug,12.0,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,delivered,51.00
3,493414,37508,new england ceramic cake server,2.0,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,5.10
4,493414,35001G,hand open shape gold,2.0,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,delivered,8.50
...,...,...,...,...,...,...,...,...,...,...
86811,505331,21912,vintage snakes & ladders,4.0,2010-04-21 12:36:00,3.75,17622.0,2010-04-21,delivered,15.00
86812,505331,22550,holiday fun ludo,4.0,2010-04-21 12:36:00,3.75,17622.0,2010-04-21,delivered,15.00
86813,505331,21888,bingo set,4.0,2010-04-21 12:36:00,3.75,17622.0,2010-04-21,delivered,15.00
86814,505331,21791,vintage heads and tails card game,12.0,2010-04-21 12:36:00,1.25,17622.0,2010-04-21,delivered,15.00


In [27]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86816 entries, 0 to 86815
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      86816 non-null  object        
 1   product_code  86816 non-null  object        
 2   product_name  86816 non-null  object        
 3   quantity      86816 non-null  float64       
 4   order_date    86816 non-null  object        
 5   price         86816 non-null  float64       
 6   customer_id   86816 non-null  object        
 7   date          86816 non-null  datetime64[ns]
 8   order_status  86816 non-null  object        
 9   amount        86816 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 6.6+ MB


# RFM Transaction

## Aggregating the transaction data by order_count and order_value from user

In [28]:
df_user = df_clean.groupby('customer_id', as_index=False).agg(order_cnt=('order_id','nunique'),max_order_date=('date','max'),total_order_value=('amount','sum'))
df_user

,customer_id,order_cnt,max_order_date,total_order_value
0,12346.0,2,2010-03-02,130.55
1,12747.0,5,2010-03-22,502.79
2,12748.0,28,2010-04-16,3395.99
3,12820.0,2,2010-02-23,706.55
4,12821.0,1,2010-04-07,128.08
...,...,...,...,...
1896,18271.0,1,2010-04-09,488.30
1897,18272.0,1,2010-03-04,635.00
1898,18276.0,1,2010-04-20,300.92
1899,18283.0,3,2010-03-28,354.42


## Make day_since_last_order column

In [29]:
today = df_clean['date'].max()
df_user['day_since_last_order'] = (today - df_user['max_order_date']).dt.days
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order
0,12346.0,2,2010-03-02,130.55,50
1,12747.0,5,2010-03-22,502.79,30
2,12748.0,28,2010-04-16,3395.99,5
3,12820.0,2,2010-02-23,706.55,57
4,12821.0,1,2010-04-07,128.08,14
...,...,...,...,...,...
1896,18271.0,1,2010-04-09,488.30,12
1897,18272.0,1,2010-03-04,635.00,48
1898,18276.0,1,2010-04-20,300.92,1
1899,18283.0,3,2010-03-28,354.42,24


In [30]:
df_user.describe()

,order_cnt,max_order_date,total_order_value,day_since_last_order
count,1901.000000,1901,1901.000000,1901.000000
mean,2.532877,2010-03-13 07:45:51.604418560,782.803413,38.676486
min,1.000000,2010-01-04 00:00:00,2.600000,0.000000
25%,1.000000,2010-02-22 00:00:00,222.500000,14.000000
50%,2.000000,2010-03-19 00:00:00,409.690000,33.000000
75%,3.000000,2010-04-07 00:00:00,801.820000,58.000000
max,52.000000,2010-04-21 00:00:00,46027.590000,107.000000
std,3.476023,NaN,1686.665444,28.160331


## Make a binning from since last order. Consist of 5 bins. Start from P20, P40, P60, P80 and Max

In [31]:
df_user['recency_score'] = pd.cut(df_user['day_since_last_order'],
                                  bins=[df_user['day_since_last_order'].min(),
                                        np.percentile(df_user['day_since_last_order'], 20),
                                        np.percentile(df_user['day_since_last_order'], 40),
                                        np.percentile(df_user['day_since_last_order'], 60),
                                        np.percentile(df_user['day_since_last_order'], 80),
                                        df_user['day_since_last_order'].max()],
                                  labels=[5, 4, 3, 2, 1],
                                  include_lowest=True).astype(int)
df_user

,customer_id,order_cnt,max_order_date,total_order_value,day_since_last_order,recency_score
0,12346.0,2,2010-03-02,130.55,50,2
1,12747.0,5,2010-03-22,502.79,30,3
2,12748.0,28,2010-04-16,3395.99,5,5
3,12820.0,2,2010-02-23,706.55,57,2
4,12821.0,1,2010-04-07,128.08,14,4
...,...,...,...,...,...,...
1896,18271.0,1,2010-04-09,488.30,12,4
1897,18272.0,1,2010-03-04,635.00,48,2
1898,18276.0,1,2010-04-20,300.92,1,5
1899,18283.0,3,2010-03-28,354.42,24,4


In [33]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1901 entries, 0 to 1900
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           1901 non-null   object        
 1   order_cnt             1901 non-null   int64         
 2   max_order_date        1901 non-null   datetime64[ns]
 3   total_order_value     1901 non-null   float64       
 4   day_since_last_order  1901 non-null   int64         
 5   recency_score         1901 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 89.2+ KB


## Make a binning from order_count

In [37]:
# Define the bin edges
bin_edges = [
    df_user['order_cnt'].min(),
    np.percentile(df_user['order_cnt'], 20),
    np.percentile(df_user['order_cnt'], 40),
    np.percentile(df_user['order_cnt'], 60),
    np.percentile(df_user['order_cnt'], 80),
    df_user['order_cnt'].max()
]

# Define the bin labels (one fewer than the number of bin edges)
bin_labels = [5, 4, 3, 2, 1]

# Use pd.cut to create the 'frequency_score' column
df_user['frequency_score'] = pd.cut(df_user['order_cnt'],
                                    bins=bin_edges,
                                    labels=bin_labels,
                                    include_lowest=True,
                                    duplicates='drop').astype(int)

# Display the resulting DataFrame
print(df_user)

ValueError: Bin labels must be one fewer than the number of bin edges